In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import norm
import numpy as np
import datetime

### Wczytywanie danych 

In [ ]:
df = pd.read_csv('D:/Projektiki/Dannyje/hotel_booking.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Przyjrzyjmy się również procentowi brakujących wartości

In [ ]:
missing_percentage = df.isnull().mean() * 100
print(missing_percentage[missing_percentage > 0])


### Analiza pierwotnych danych oraz ich przygotowanie
Dane zawierają 36 kolumn i 119390 wpisów

* `hotel` -- typ hotelu;
* `is_canceled` -- status anulowania (atrybut docelowy);
* `lead_time` -- czas przed przyjazdem;
* `arrival_date_year`, `arrival_date_month`, `arrival_date_week_number`, `arrival_date_day_of_month` -- data przyjazdu;
* `stays_in_weekend_nights`, `stays_in_week_nights` -- liczba noclegów w weekendy i dni powszednie;
* `adults`, `children`, `babies` -- liczba osób dorosłych, dzieci i niemowląt;
* `meal` -- rodzaj posiłku;
* `country` --  kraj klienta;
* `market_segment` -- segment rynku;
* `distribution_channel` -- kanał dystrybucji;
* `is_repeated_guest` -- stały klient;
* `previous_cancellations`, `previous_bookings_not_cancled` -- liczba wcześniejszych anulowanych i nieodwołanych rezerwacji;
* `reserved_room_type`, `assigned_room_type` -- typ zarezerwowanego i przydzielonego pokoju;
* `booking_changes` -- liczba zmian rezerwacji;
* `deposit_type` -- typ depozytu;
* `agent`, `company` -- identyfikatory agenta i firmy;
* `days_in_waiting_list` -- liczba dni na liście oczekujących;
* `customer_type` -- typ klienta;
* `adr` -- średnia stawka dzienna;
* `required_car_parking_spaces` -- liczba wymaganych miejsc parkingowych;
* `total_of_special_requests` -- całkowita liczba zamówień specjalnych;
* `reservation_status`, `reservation_status_date` -- status rezerwacji i data statusu;

##### W niektórych kolumnach istnieją brakujące wartości
`children`, `country`, `agent`, `company` trzeba dopełnić brakujące wartości

##### Ponadto, aby uprościć proces pracy z danymi, można usunąć niepotrzebne kolumny:
`name`, `email`, `phone-number`, `credit_card`



In [ ]:
df['children'].fillna(0, inplace=True)
df['country'].fillna('Unknown', inplace=True)
df['agent'].fillna(0, inplace=True)
df['company'].fillna(0, inplace=True)

df = df.drop(columns=['name', 'email', 'phone-number', 'credit_card'])

Sprawdźmy, czy nie ma duplikatów.

In [ ]:
len(df[df.duplicated()])

Usuniemy duplikaty

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['is_canceled'] = df['is_canceled'].astype(bool)
df['is_repeated_guest'] = df['is_repeated_guest'].astype(bool)

Konwertowanie nazw miesięcy na wartości liczbowe

In [ ]:
df['arrival_date_month'] = df['arrival_date_month'].map({'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7,'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12})
     

In [ ]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

Zmienimy typ danych dla kolumny `children`

In [ ]:
df[['children']] = df[['children']].astype('int')

### EDA

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='is_canceled', hue='is_canceled', data=df, palette="Set2")
plt.title('Rozkład zmiennej docelowej (is_canceled)')
plt.show()

cancel_rate = df['is_canceled'].mean() * 100
print(f"Procent anulowanych rezerwacji: {cancel_rate:.2f}%")

Widzimy wyraźny brak równowagi klas w kolumnie `is_canceled` 

*Przyjrzyjmy się rezerwacjom w różnych typach hoteli*

In [ ]:
plt.figure(figsize=(12, 6))  

ax1 = sns.countplot(x='hotel', hue='is_canceled', data=df, palette="Set2")

ax1.legend(bbox_to_anchor=(1, 1), title="Status rezerwacji")

plt.title('Status rezerwacji w różnych hotelach', size=20)
plt.xlabel('Hotel', fontsize=16)
plt.ylabel('Liczba rezerwacji', fontsize=16)
plt.legend(['Nie anulowanа', 'Anulowana'], title='Status rezerwacji', fontsize=12)
plt.show()

================================

In [ ]:
df['month']=df['reservation_status_date'].dt.month
ax1 = sns.countplot(x='month', hue='is_canceled', data= df, palette = 'Set2')
legend_lebels,_ = ax1.get_legend_handles_labels()
plt.title('Rezerwacja/miesiąć', size = 20)
plt.xlabel('miesiąc')
plt.ylabel('Liczba rezerwacji')
plt.legend(['Nie anulowanа','Anulowana'], title='Status rezerwacji')
plt.show()

#### Macierz korelacji cech numerycznych

In [ ]:
numeric_df = df.select_dtypes(include='number')
corr_matrix = numeric_df.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, cmap="Greens", annot=True, fmt=".1f")
plt.title('Macierz korelacji cech numerycznych')
plt.show()

================================

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 9))
sns.countplot(x='arrival_date_year', hue='is_canceled', data=df, palette = 'Set2')

plt.title('arrival_date_year distribution')
plt.xlabel('Years')
plt.ylabel('Quantity')

plt.show()

In [ ]:
print(df['adults'].value_counts())
print(df['children'].value_counts()) 
print(df['babies'].value_counts())

Najczęściej rezerwowany jest pokój dla 2 osób dorosłych.

In [ ]:
mu = df['arrival_date_week_number'].mean()
std = df['arrival_date_week_number'].std()

plt.figure(figsize=(12, 9))
sns.histplot(df['arrival_date_week_number'], bins=20, color='darkgreen', kde=False)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p * len(df['arrival_date_week_number']) * (xmax - xmin) / 20, 'r--', linewidth=2)

plt.show()


Dane w kolumnie `arrival_date_week_number` mają prawie normalny rozkład.

In [ ]:
mu = df['booking_changes'].mean()
std = df['booking_changes'].std()

# Построить гистограмму данных
plt.figure(figsize=(12, 9))
sns.histplot(df['booking_changes'], color='darkgreen', kde=False)

# Добавить нормальное распределение
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p * len(df['booking_changes']) * (xmax - xmin) / 20, 'r--', linewidth=2)


plt.title('booking_changes distribution', fontsize=16)
plt.xlabel('Liczba zmian rezerwacji', fontsize=14)
plt.xticks(fontsize=12)
plt.grid(True)

plt.show()


W kolumnie `booking_changes` widzimy rozkład wykładniczy

##### Analiza krajów przybycia 
Istnieje wiele krajów z niewielką liczbą odwiedzin w kolumnie `country`, więc zachowajmy 12 krajów i przypiszmy wartość "inne" pozostałym.

In [ ]:
countries = ['PRT', 'FRA', 'GBR', 'ESP', 'DEU', 'ITA', 'IRL', 'BEL', 'BRA', 'NLD', 'USA', 'CHE']
df['country'] = df['country'].apply(lambda x: 'others' if x not in countries else x)


total_orders = df['country'].value_counts()
canceled_orders = df[df['is_canceled'] == 1]['country'].value_counts()


df_stacked = pd.DataFrame({
    'Łączna liczba rezerwacji': total_orders,
    'Anulowane rezerwacji': canceled_orders
}).fillna(0)


df_stacked.plot(kind='bar', stacked=True, figsize=(12, 9), color=['darkgreen', 'darkred'], alpha=0.9)
plt.title('Rezerwacji ogółem i anulowane według kraju', fontsize=16)
plt.xlabel('Kraje', fontsize=14)
plt.ylabel('Liczba', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title='Status rezerwacji', fontsize=12)
plt.grid(axis='y', linestyle='--', linewidth=0.7)

plt.show()

Widać, że więcej rezerwacji pochodzi od klientów z Portugalii. Dlatego więcej anulowanych rezerwacji również pochodzi od portugalskich klientów.